## Package imports

In [2]:
from enum import Enum
import pandas as pd
import ollama

## Text cleaning

- Cleared the print area to remove warnings produced after reading from spreadsheets

### Helper functions

In [3]:
# Insert the list of levels
def insert_levels(df):
    levels = df.index.str.len()
    df.insert(0, "Level", levels)

# Insert the list of parents given an industry classification
def insert_parents(df):
    parents = []
    history = []
    prev = {"Level": 0, "Code": ""}

    # Iterate through the dataframe to deduce the parent
    for row in df.itertuples():
        prev_level = len(history)
        current_level = row.Level
        current_code = row.Index
        
        ## If the current level is '1'
        if current_level == 1:
            current_parent = ""
            history = [current_code]
            
        ## If the current level is lower than the previous level
        elif current_level > prev_level:
            current_parent = history[prev_level - 1]
            history.append(current_code)
            
        ## If the current level is higher than the previous level
        elif current_level < prev_level:
            current_parent = history[(current_level - 1) - 1]
            del history[current_level - 1:]
            history.append(current_code)
        
        else:
            current_parent = history[(current_level - 1) - 1]
        
        parents.append(current_parent)
        prev.update({"Level": current_level, "Code": current_code})
        
    # Insert the list as a column
    df.insert(1, "Parent", parents)

# Usual string cleaning techniques
def default_clean(df):
    # Normalisation (lowercase strings)
    # lower = lambda x: x.lower() if isinstance(x, str) else x
    
    ## If the dataframe has ISIC code data
    # if "ISIC code" in df:
    #     if "Parent" in df:
    #         df.iloc[:, 2:-1] = df.iloc[:, 2:-1].map(lower)
    #     else:
    #         df.iloc[:, 1:-1] = df.iloc[:, 1:-1].map(lower)
        
    ## If the dataframe doesnt' have ISIC code data
    # else:
    #     df.loc[:, "Description":] = df.loc[:, "Description":].map(lower)
    
    # Replace punctuation with empty strings
    # df.loc[:, "Description":] = df.loc[:, "Description":].replace(r"[^\w\s]+", " ", regex=True)
    
    # Replace newlines and multiple spaces with whitespaces
    df.replace([r"\n", r" +"], " ", regex=True, inplace=True)
    
    # Strip leading and trailing whitespaces
    df.loc[:, "Description":] = df.loc[:, "Description":].map(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Replace 'and or' with 'or'
    df.replace(r"and/or", "or", regex=True, inplace=True)

### Cleaning ISIC data

In [4]:
# Read the CSV file
ISIC_df = pd.read_csv("data/ISIC Rev. 4.csv")

# Set the 'Code' column as the index
ISIC_df = ISIC_df.set_index("Code")

# Add a 'Level' column
insert_levels(ISIC_df)

# Add a 'Parent' column
insert_parents(ISIC_df)

# Extra cleaning techniques
default_clean(ISIC_df)

ISIC_df.head()

,Level,Parent,Description
Code,,,
A,1,,"Agriculture, forestry and fishing"
01,2,A,"Crop and animal production, hunting and relate..."
011,3,01,Growing of non-perennial crops
0111,4,011,"Growing of cereals (except rice), leguminous c..."
0112,4,011,Growing of rice


### Cleaning NACE data

In [5]:
# Read the Excel file
NACE_df = pd.read_excel("data/NACE Rev. 2.xlsx")

# Replace NaN values with empty strings
NACE_df = NACE_df.fillna("")

# Set the 'Code' column as the index
NACE_df = NACE_df.set_index("Code")

# Rename columns
NACE_df = NACE_df.rename(columns={
    "This item includes": "Examples",
    "This item excludes": "Exclusions",
    "Reference to ISIC Rev. 4": "ISIC code"
})

# Concatenate the columns "This item also includes" and "Definition"
NACE_df["Examples"] += " " + NACE_df["This item also includes"]

# Drop columns
NACE_df = NACE_df.drop(columns=["Order", "Rulings", "This item also includes"])

# Extra cleaning techniques
default_clean(NACE_df)

NACE_df.head()

,Level,Parent,Description,Examples,Exclusions,ISIC code
Code,,,,,,
A,1,,"AGRICULTURE, FORESTRY AND FISHING",This section includes the exploitation of vege...,,A
01,2,A,"Crop and animal production, hunting and relate...","This division includes two basic activities, n...",Agricultural activities exclude any subsequent...,01
01.1,3,01,Growing of non-perennial crops,This group includes the growing of non-perenni...,,011
01.11,4,01.1,"Growing of cereals (except rice), leguminous c...",This class includes all forms of growing of ce...,"This class excludes: - growing of rice, see 01...",0111
01.12,4,01.1,Growing of rice,This class includes: - growing of rice (includ...,,0112


### Cleaning WZ data

In [6]:
# Read the Excel file
WZ_df = pd.read_excel("data/WZ Issue 2008.xls", sheet_name="Content")

# Rename columns
WZ_df = WZ_df.rename(columns={"Code WZ 2008": "Code", "Title": "Description"})

# Set the 'Code' column as the index
WZ_df = WZ_df.set_index("Code")

# Add a 'Parent' column
insert_parents(WZ_df)

# Remove columns
WZ_df = WZ_df.drop(columns="Unit(s) of measure")

# Extra cleaning techniques
default_clean(WZ_df)

WZ_df.head()

,Level,Parent,Description
Code,,,
A,1,,"Agriculture, forestry and fishing"
01,2,A,"Crop and animal production, hunting and relate..."
01.1,3,01,Growing of non-perennial crops
01.11,4,01.1,"Growing of cereals (except rice), leguminous c..."
01.11.0,5,01.11,"Growing of cereals (except rice), leguminous c..."


### Cleaning SSIC v1 data

In [7]:
# Read the Excel file
SSIC_v1_df = pd.read_excel("data/SSIC 2020 v1.xlsx")

# Drop columns
SSIC_v1_df = SSIC_v1_df.drop(columns=["ISIC Rev. 4 Part", "ISIC Rev. 4 Title"])

# Rename columns
SSIC_v1_df = SSIC_v1_df.rename(columns={
    "SSIC 2020 Title": "Description",
    "SSIC 2020": "Code",
    "ISIC Rev. 4": "ISIC code"
})

# Set the 'Code' column as the index
SSIC_v1_df = SSIC_v1_df.set_index("Code")

# Add a 'Level' column
insert_levels(SSIC_v1_df)

# Extra cleaning techniques
default_clean(SSIC_v1_df)

SSIC_v1_df.head()

,Level,Description,ISIC code
Code,,,
01111,5,Growing of leafy and fruit vegetables,113
01112,5,Growing of mushrooms,113
01113,5,Growing of root crops,113
01119,5,Growing of food crops (non-hydroponics) n.e.c.,111
01119,5,Growing of food crops (non-hydroponics) n.e.c.,112


### Cleaning SSIC v2 data

In [8]:
# Read the Excel file
SSIC_v2_df = pd.read_excel("data/SSIC 2020 v2.xlsx")

# Drop columns
SSIC_v2_df = SSIC_v2_df.drop(columns=["Cross References", "Groups Classified Under this Code"])

# Replace '<Blank>' with empty strings
SSIC_v2_df = SSIC_v2_df.replace(to_replace="<Blank>", value="")

# Rename columns
SSIC_v2_df = SSIC_v2_df.rename(columns={
    "SSIC 2020 Title": "Description",
    "SSIC 2020": "Code",
    "Detailed Definitions": "Definition",
    "Examples of Activities Classified Under this Code": "Examples"
})

# Set the 'Code' column as the index
SSIC_v2_df = SSIC_v2_df.set_index("Code")

# Add a 'Level' column
insert_levels(SSIC_v2_df)

# Add a 'Parent' column
insert_parents(SSIC_v2_df)

# Extra cleaning techniques
default_clean(SSIC_v2_df)

SSIC_v2_df.head()

,Level,Parent,Description,Definition,Examples
Code,,,,,
A,1,,AGRICULTURE AND FISHING,,
01,2,A,AGRICULTURE AND RELATED SERVICE ACTIVITIES,,
011,3,01,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",,
0111,4,011,Growing of Food Crops (Non-Hydroponics),,
01111,5,0111,Growing of leafy and fruit vegetables,This sub-class includes the cultivation of lea...,


## Pre-processing

### Using spaCy

In [9]:
import spacy
from spacy import displacy
from spacy.language import Language

nlp = spacy.load("en_core_web_md")

# Add the function to the pipeline
config = {"punct_chars": ['•', '\n']}
nlp.add_pipe("sentencizer", config=config, before="parser")

KeyboardInterrupt: 

In [ ]:
instruction = "Only output the answer for the following query WITHOUT any introductory or conclusion text. Refer to the guidelines and examples to supplement your understanding of the query. Do NOT reply directly to me."

query = "Produce a single-line string with all activities separated by '|' in the given input."

guidelines = [
    "Ensure all processes within the input are mentioned in the final result.",
    "If an input involving processes 'X', 'Y' and 'Z' is of the form 'X, Y and Z of (noun)', the final result should be 'X of (noun)|Y of (noun)|Y of (noun)'.",
    "Tthe previous guideline should still be followed even if there are no conjunctions such as 'and' between processes. The input format provided in the previous guideline is not exhaustive.",
    "'etc.' and other such abbrievations illustrating a non-exhaustive list of processes should not be included in the final result."
]

guidelines_str = "\n".join([f"{i + 1}. {text}" for i, text in enumerate(guidelines)])

examples = [
    "'plating, anodising etc. of metals' should be 'plating of metals|anodising of metals'.",
    "'deburring, sandblasting, tumbling, cleaning of metals' should be 'deburring of metals|sandblasting of metals|tumbling of metals|cleaning of metals'."
]

examples_str = "\n".join([f"{i + 1}. {text}" for i, text in enumerate(examples)])

def message(input):
    return f'''{instruction}

    Query: {query}

    Guidelines to follow:
    {guidelines_str}

    Examples:
    {examples_str}

    Input: "{input}"'''

In [ ]:
# content = message("""This class includes:
# - production of dried, salted or smoked meat
# - production of meat products:
#   . sausages, salami, puddings, ""andouillettes"", saveloys, bolognas, pâtés, rillettes, boiled ham
# """)

# response = ollama.chat(
#     model='llama3',
#     messages=[
#         {
#             'role': 'user',
#             'content': content,
#         },
#     ],
#     options={
#         "temperature": 0.1
#     }
# )

# print(response["message"]["content"])

In [10]:
# Save the preprocessed data to a new CSV file
from pathlib import Path  

def to_csv(df, basename):
    filepath = Path(f"data/exports/{basename}")  
    filepath.parent.mkdir(parents=True, exist_ok=True)  

    df.to_csv(filepath)

to_csv(NACE_df, "NACE.csv")

In [ ]:
# class IndustryCodeType(Enum):
#     ISIC, NACE, WZ, SSIC = range(4)

# class IndustryCode:
#     def __init__(self, type, value):
#         self.type = type
#         self.value = value

# class Company:
#     def __init__(self, name, desc, code):
#         self.name = name
#         self.desc = desc
#         self.code = code